In [1]:
import pickle
import os

from sklearn.metrics import r2_score
import pandas as pd


# モデルの読み込み

In [2]:
rf = pickle.load(open(os.path.join('regressor.pkl'),'rb'))

/Users/arimitsushuntarou/Downloads/yes/envs/bikeflask/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/arimitsushuntarou/Downloads/yes/envs/bikeflask/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
#データの読み込み
train_data = pd.read_csv("../data/機械学習用/train_data.csv",index_col=0)
model_data = pd.read_csv('../data/アップロード用/model_data.csv',index_col=0)
info_data = pd.read_csv('../data/アップロード用/bike_info.csv',index_col=0)
#サンプルサイズが少ないモデルを抽出
all_data = info_data.merge(model_data,on='model_number',how='left')
brand_count = all_data.groupby('brand').size().sort_values(ascending=True)
#下位10メーカーを抽出(上のcountデータがすでに照準のため上から９つを抽出する)
low_10 = brand_count.index[0:10]
model_data = model_data[~model_data['brand'].isin(low_10)]
#このメーカーデータを使ってtrainデータに結合
train_data = train_data.merge(model_data,on='model_number',how='left')
#先ほど外れ値に指定したデータはnullになるため、dropnaをすればok
train_data.dropna(inplace=True)
test_data = pd.read_csv('../data/機械学習用/test_data.csv',index_col=0)
test_data=test_data.merge(model_data,on='model_number',how='left')
test_data.dropna(inplace=True)
#ダミーエンコーディング
#エンコーディングしたい列
encoding_target = ['color','brand','type']
train_data=pd.get_dummies(train_data,drop_first=True,columns=encoding_target)
#ラベルエンコーディングを反映
test_data=pd.get_dummies(test_data,drop_first=True,columns=encoding_target)
X_test=test_data.iloc[:,2:].drop(columns=['name','model_number','store_id']).values
y_test = test_data['cost'].values

In [4]:
rf

RandomForestRegressor(bootstrap=False, max_depth=34,
                      max_features=0.5967285924743135, max_leaf_nodes=751,
                      min_samples_leaf=2, min_samples_split=3, n_estimators=85,
                      random_state=0)

In [5]:
#チューニングをしたrandomforestモデルが持って来れる
predict_test = rf.predict(X_test)
r2_score(y_true=y_test,y_pred=predict_test)

0.9730308296131362

In [6]:
test_data

,id,cost,model_number,model_year,mileage,guarantee_period,store_id,name,displacement,color_ピンク,...,brand_Husqvarna,brand_KAWASAKI,brand_KTM,brand_SUZUKI,brand_Triumph,brand_YAMAHA,type_オフロード/モタード,type_スクーター/ビッグスクーター,type_スポーツ/ツアラー,type_ネイキッド
0,2100006178999,388000,2BK-DN11A,2021.0,5034.0,5,63,GSX250R,250.0,False,...,False,False,False,True,False,False,False,False,True,False
1,2100005344975,1480000,2BL-ZR900C,2020.0,8354.0,5,50,Z900RS,950.0,False,...,False,True,False,False,False,False,False,False,False,True
2,2100006036176,148000,JBH-SA52J,2016.0,11896.0,3,50,VOX,50.0,False,...,False,False,False,False,False,True,False,True,False,False
3,2100005759090,1428000,JD5,2010.0,15498.0,3,27,FLSTN1580,1580.0,False,...,False,False,False,False,False,False,False,False,False,False
4,2100006282986,1100000,2BL-NC42,2020.0,12067.0,3,65,CB400 SUPER FOUR HYPER VTEC Revo,400.0,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,2100005863018,498000,2BL-RH13J,2018.0,3371.0,5,46,YZFR3,320.0,False,...,False,False,False,False,False,True,False,False,False,True
1118,2100005567510,588000,2BL-NC59,2021.0,21834.0,1,13,GB350,350.0,False,...,False,False,False,False,False,False,False,False,False,True
1119,2100006115840,2898000,2BL-FLC,2018.0,8920.0,5,36,FLTRX1750Road Glide Special,1750.0,False,...,False,False,False,False,False,False,False,False,False,False
1120,2100005856706,1368000,LC3,2011.0,14216.0,3,1,XL1200X,1200.0,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
test_data['guarantee_period'].dtype

dtype('int64')

In [7]:
X_test

array([[2021.0, 5034.0, 5, ..., False, True, False],
       [2020.0, 8354.0, 5, ..., False, False, True],
       [2016.0, 11896.0, 3, ..., True, False, False],
       ...,
       [2018.0, 8920.0, 5, ..., False, False, False],
       [2011.0, 14216.0, 3, ..., False, False, False],
       [2010.0, 21583.0, 3, ..., False, False, False]], dtype=object)

In [8]:
data=pd.Series(data=[1,2000,111,2020,'赤',10000,2,1,'HONDA','CBR1000RR','スポーツ/ツアラー','1000'],index=all_data.columns)
df_t=pd.DataFrame(columns=all_data.columns)
df_t.loc[1]=data

In [9]:
df_t

,id,cost,model_number,model_year,color,mileage,guarantee_period,store_id,brand,name,type,displacement
1,1,2000,111,2020,赤,10000,2,1,HONDA,CBR1000RR,スポーツ/ツアラー,1000


In [10]:
df_t=pd.get_dummies(df_t,drop_first=True,columns=encoding_target)

In [85]:
df_t

,id,cost,model_number,model_year,mileage,guarantee_period,store_id,name,displacement
1,1,2000,111,2020,10000,2,1,CBR1000RR,1000


In [77]:

X_df_t = df_t.iloc[:,2:].drop(columns=['name','model_number','store_id']).values

In [79]:
X_df_t

array([[2020, 10000, 2, '1000']], dtype=object)

In [10]:
rf.predict(X_df_t)

NameError: name 'X_df_t' is not defined

# インプットを受け取るためのDBを作成

In [10]:
import sqlite3
import os
conn = sqlite3.connect('inputdata.sqlite')
c = conn.cursor()
c.execute('DROP TABLE IF EXISTS input_db')
c.execute('CREATE TABLE input_db (model_year FLOAT, mileage FLOAT, guarantee_period FLOAT, color TEXT, brand TEXT, type TEXT ,displacement FLOAT )')

In [11]:
c.execute('INSERT INTO input_db (model_year, mileage, guarantee_period,color,brand,type,displacement) VALUES'\
          '(?,?,?,?,?,?,?)',(2022,1100,3,"赤","HONDA","スポーツ/ツアラー",1000) )
conn.commit()
conn.close()

In [12]:
conn = sqlite3.connect('inputdata.sqlite')
c = conn.cursor()

In [13]:
c.execute('SELECT * FROM input_db ')
results = c.fetchall()
conn.close()
print(results)

[(2022.0, 1100.0, 3.0, '赤', 'HONDA', 'スポーツ/ツアラー', 1000.0)]
